In [1]:
pip install PySastrawi

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SUCI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SUCI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SUCI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Preprocessing using experiment I

In [6]:
df = pd.read_excel("Dataset.xlsx")
colom = df.columns

print (colom)

harga = df['harga'].tolist()
terjual = df['terjual'].tolist()
rating= df['rating'].tolist()
namatoko= df['namatoko'].tolist()
kategori = df['kategori'].tolist()
subkategori = df['subkategori'].tolist()
marketplace = df['marketplace'].tolist()

nmproduk = df['namaproduk']
desc = df['deskripsi']

def capital_space(text):
     text = re.sub(r"([a-z])([A-Z])", r"\1 \2",str(text))
     text = re.sub(r"\d+", "  ", text)
     return text
capitaldelproduct = nmproduk.apply(capital_space)
capitaldeldesc = desc.apply(capital_space)
# capitaldelproduct.tolist()
# capitaldeldesc.tolist()


##---Case Folding---
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

lowercaseprd = capitaldelproduct.apply(clean_lower)
lowercasedesc = capitaldeldesc.apply(clean_lower)
# lowercaseprd.tolist()
# lowercasedesc.tolist()

##---Remove Punctuation---
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('  ', text)
    text = clean_symbol.sub('  ', text)
    return text


removepuncprd = lowercaseprd.apply(clean_punct)
removepuncdesc = lowercasedesc.apply(clean_punct)
# removepuncprd.tolist()
# removepuncdesc.tolist()

##--Remove Stopwords---
dict = pd.read_excel("stoplist-with-analysis-terms.xlsx")
stopwordid = stopwords.words('indonesian')
stopwordeng = stopwords.words('english')
stoplist = dict['stoplist'].tolist()
stoplist= stoplist+stopwordid+stopwordeng  
  
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stoplist)
    
    return text
stopwordsproduct = removepuncprd.apply(clean_stopwords)
stopwordsdesc = removepuncdesc.apply(clean_stopwords)
# stopwordsproduct.tolist()
# stopwordsdesc.tolist()

# print(stopwordsdesc.tolist())


##---StemmingSastrawi---
factory = StemmerFactory()
stemmer = factory.create_stemmer()
ps = PorterStemmer()

def stemming(text):
    text = ' '.join(stemmer.stem(word) for word in text.split() if word in text)
    text = ' '.join(ps.stem(word) for word in text.split() if word in text)
    return text

stemmingprod = stopwordsproduct.apply(stemming)
stemmingdesc = stopwordsdesc.apply(stemming)
produk = stemmingprod.tolist()
deskripsi = stemmingdesc.tolist()

#Merge Data produk  dan Desc
df = pd.DataFrame(list(zip(produk, deskripsi)), columns =['namaproduk', 'desc']) 

df['harga'] = harga
df['terjual'] = terjual
df['rating'] = rating
df['namatoko'] = namatoko
df['kategori'] = kategori
df['subkategori'] = subkategori
df['marketplace'] = marketplace
df.to_csv('dataset-after-preprocess-experiment1.csv')

Index(['namaproduk', 'harga', 'terjual', 'rating', 'deskripsi', 'namatoko',
       'kategori', 'subkategori', 'marketplace'],
      dtype='object')


In [ ]:
#Preprocessing using experiment II

In [10]:
df = pd.read_excel("Dataset.xlsx")
colom = df.columns

print (colom)

harga = df['harga'].tolist()
terjual = df['terjual'].tolist()
rating= df['rating'].tolist()
namatoko= df['namatoko'].tolist()
kategori = df['kategori'].tolist()
subkategori = df['subkategori'].tolist()
marketplace = df['marketplace'].tolist()

nmproduk = df['namaproduk']
desc = df['deskripsi']

def capital_space(text):
    text = re.sub(r"([a-z])([A-Z])_\s-", r"\1 \2",str(text))
    text = re.sub(r"\d+", "  ", text)
    return text
capitaldelproduct = nmproduk.apply(capital_space)
capitaldeldesc = desc.apply(capital_space)
# capitaldelproduct.tolist()
# capitaldeldesc.tolist()


##---Case Folding---
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

lowercaseprd = capitaldelproduct.apply(clean_lower)
lowercasedesc = capitaldeldesc.apply(clean_lower)
# lowercaseprd.tolist()
# lowercasedesc.tolist()

##---Remove Punctuation---
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('  ', text)
    text = clean_symbol.sub(' ', text)
    return text

removepuncprd = lowercaseprd.apply(clean_punct)
removepuncdesc = lowercasedesc.apply(clean_punct)
# removepuncprd.tolist()
# removepuncdesc.tolist()

##--Remove Stopwords---
dict = pd.read_excel("stoplist.xlsx")
stopwordid = stopwords.words('indonesian')
stopwordeng = stopwords.words('english')
stoplist = dict['stoplist'].tolist()
stoplist= stoplist+stopwordid+stopwordeng  
  
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stoplist) 
    return text

stopwordsproduct = removepuncprd.apply(clean_stopwords)
stopwordsdesc = removepuncdesc.apply(clean_stopwords)

##---StemmingSastrawi---
factory = StemmerFactory()
stemmer = factory.create_stemmer()
ps = PorterStemmer()

def stemming(text):
    text = ' '.join(stemmer.stem(word) for word in text.split() if word in text)
    text = ' '.join(ps.stem(word) for word in text.split() if word in text)
    return text

stemmingprod = stopwordsproduct.apply(stemming)
stemmingdesc = stopwordsdesc.apply(stemming)
produk = stemmingprod.tolist()
deskripsi = stemmingdesc.tolist()

#Merge Data produk  dan Desc
df = pd.DataFrame(list(zip(produk, deskripsi)), columns =['namaproduk', 'desc']) 

df['harga'] = harga
df['terjual'] = terjual
df['rating'] = rating
df['namatoko'] = namatoko
df['kategori'] = kategori
df['subkategori'] = subkategori
df['marketplace'] = marketplace
df.to_csv('dataset-after-preprocess-experiment2.csv') 

Index(['namaproduk', 'harga', 'terjual', 'rating', 'deskripsi', 'namatoko',
       'kategori', 'subkategori', 'marketplace'],
      dtype='object')


In [ ]:
#Preprocessing using experiment III

In [11]:
df = pd.read_excel("Dataset.xlsx")
colom = df.columns

print (colom)

harga = df['harga'].tolist()
terjual = df['terjual'].tolist()
rating= df['rating'].tolist()
namatoko= df['namatoko'].tolist()
kategori = df['kategori'].tolist()
subkategori = df['subkategori'].tolist()
marketplace = df['marketplace'].tolist()

nmproduk = df['namaproduk']
desc = df['deskripsi']

def capital_space(text):
     text = re.sub(r"([a-z])([A-Z])_\s-", r"\1 \2",str(text))
     text = re.sub(r"\d+", "  ", text)
     return text
capitaldelproduct = nmproduk.apply(capital_space)
capitaldeldesc = desc.apply(capital_space)

##---Case Folding---
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

lowercaseprd = capitaldelproduct.apply(clean_lower)
lowercasedesc = capitaldeldesc.apply(clean_lower)

##---Remove Punctuation---
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('  ', text)
    text = clean_symbol.sub('  ', text)
    return text


removepuncprd = lowercaseprd.apply(clean_punct)
removepuncdesc = lowercasedesc.apply(clean_punct)

#Merge Data produk  dan Desc
df = pd.DataFrame(list(zip(removepuncprd, removepuncdesc)), columns =['namaproduk', 'desc']) 

df['harga'] = harga
df['terjual'] = terjual
df['rating'] = rating
df['namatoko'] = namatoko
df['kategori'] = kategori
df['subkategori'] = subkategori
df['marketplace'] = marketplace
df.to_csv('dataset-after-preprocess-experiment3.csv') 

Index(['namaproduk', 'harga', 'terjual', 'rating', 'deskripsi', 'namatoko',
       'kategori', 'subkategori', 'marketplace'],
      dtype='object')
